### Import libraries and create DataFrame from csv 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("RB_Data_2025.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,ATT,YDS,Y/A,LG,20+,TD,REC,TGT,YDS.1,Y/R,TD.1,G,FPTS,FPTS/G
0,1,Jahmyr Gibbs (DET),250,1412,5.6,70,13,16,52,63,517,9.9,4,17,362.9,21.3
1,2,Saquon Barkley (PHI),345,2005,5.8,72,17,13,33,43,278,8.4,2,16,355.3,22.2
2,3,Bijan Robinson (ATL),304,1456,4.8,37,5,14,61,72,431,7.1,1,17,341.7,20.1
3,4,Derrick Henry (BAL),325,1921,5.9,87,19,16,19,22,193,10.2,2,17,336.4,19.8
4,5,De'Von Achane (MIA),203,907,4.5,61,5,6,78,87,592,7.6,6,17,299.9,17.6
5,6,Josh Jacobs (GB),301,1329,4.4,38,7,15,36,43,342,9.5,1,17,293.1,17.2
6,7,Kyren Williams (LAR),316,1299,4.1,30,2,14,34,40,182,5.4,2,16,272.1,17.0
7,8,James Cook (BUF),207,1009,4.9,65,6,16,32,38,258,8.1,2,16,266.7,16.7
8,9,Alvin Kamara (NO),228,950,4.2,24,3,6,68,89,543,8.0,2,14,265.3,19.0
9,10,Chase Brown (CIN),229,990,4.3,40,8,7,54,65,360,6.7,4,16,255.0,15.9


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [3]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/A', 'LG', 'Y/R', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,Y/A,LG,Y/R,G,FPTS,FPTS/G,ATT/game,YDS/game,20+/game,TD/game,REC/game,TGT/game,YDS.1/game,TD.1/game
0,1,Jahmyr Gibbs (DET),5.6,70,9.9,17,362.9,21.3,14.7,83.1,0.8,0.9,3.1,3.7,30.4,0.2
1,2,Saquon Barkley (PHI),5.8,72,8.4,16,355.3,22.2,21.6,125.3,1.1,0.8,2.1,2.7,17.4,0.1
2,3,Bijan Robinson (ATL),4.8,37,7.1,17,341.7,20.1,17.9,85.6,0.3,0.8,3.6,4.2,25.4,0.1
3,4,Derrick Henry (BAL),5.9,87,10.2,17,336.4,19.8,19.1,113.0,1.1,0.9,1.1,1.3,11.4,0.1
4,5,De'Von Achane (MIA),4.5,61,7.6,17,299.9,17.6,11.9,53.4,0.3,0.4,4.6,5.1,34.8,0.4
5,6,Josh Jacobs (GB),4.4,38,9.5,17,293.1,17.2,17.7,78.2,0.4,0.9,2.1,2.5,20.1,0.1
6,7,Kyren Williams (LAR),4.1,30,5.4,16,272.1,17.0,19.8,81.2,0.1,0.9,2.1,2.5,11.4,0.1
7,8,James Cook (BUF),4.9,65,8.1,16,266.7,16.7,12.9,63.1,0.4,1.0,2.0,2.4,16.1,0.1
8,9,Alvin Kamara (NO),4.2,24,8.0,14,265.3,19.0,16.3,67.9,0.2,0.4,4.9,6.4,38.8,0.1
9,10,Chase Brown (CIN),4.3,40,6.7,16,255.0,15.9,14.3,61.9,0.5,0.4,3.4,4.1,22.5,0.2


### Calculate the correlations for the final stats across different conditions

In [4]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/A,0.66,0.34,0.24,0.63,0.47
LG,0.82,0.70,0.36,0.35,0.56
Y/R,0.56,0.22,0.05,0.26,0.27
ATT/game,0.94,0.92,0.85,0.60,0.83
YDS/game,0.95,0.94,0.86,0.75,0.87
20+/game,0.76,0.75,0.65,0.58,0.68
TD/game,0.87,0.87,0.82,0.65,0.80
REC/game,0.85,0.83,0.55,0.07,0.57
TGT/game,0.85,0.82,0.55,0.06,0.57
YDS.1/game,0.83,0.80,0.53,0.17,0.58


### Assign the weights for the final stats

In [5]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/A,0.47,0.22,0.07
LG,0.56,0.31,0.08
Y/R,0.27,0.07,0.06
ATT/game,0.83,0.68,0.14
YDS/game,0.87,0.77,0.15
20+/game,0.68,0.47,0.08
TD/game,0.80,0.64,0.13
REC/game,0.57,0.33,0.08
TGT/game,0.57,0.33,0.08
YDS.1/game,0.58,0.34,0.08


### Multiply the assigned weights to the final stats and calculate the score

In [6]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,Y/A_weighted,LG_weighted,ATT/game_weighted,YDS/game_weighted,20+/game_weighted,TD/game_weighted,REC/game_weighted,TGT/game_weighted,YDS.1/game_weighted,TD.1/game_weighted
0,1,Jahmyr Gibbs (DET),21.3,0.4,5.3,2.0,12.1,0.1,0.1,0.2,0.3,2.3,0.0
1,2,Saquon Barkley (PHI),22.2,0.4,5.4,2.9,18.3,0.1,0.1,0.2,0.2,1.3,0.0
2,3,Bijan Robinson (ATL),20.1,0.3,2.8,2.4,12.5,0.0,0.1,0.3,0.3,2.0,0.0
3,4,Derrick Henry (BAL),19.8,0.4,6.6,2.6,16.5,0.1,0.1,0.1,0.1,0.9,0.0
4,5,De'Von Achane (MIA),17.6,0.3,4.6,1.6,7.8,0.0,0.1,0.4,0.4,2.7,0.0
5,6,Josh Jacobs (GB),17.2,0.3,2.9,2.4,11.4,0.0,0.1,0.2,0.2,1.6,0.0
6,7,Kyren Williams (LAR),17.0,0.3,2.3,2.7,11.8,0.0,0.1,0.2,0.2,0.9,0.0
7,8,James Cook (BUF),16.7,0.3,4.9,1.8,9.2,0.0,0.1,0.2,0.2,1.2,0.0
8,9,Alvin Kamara (NO),19.0,0.3,1.8,2.2,9.9,0.0,0.1,0.4,0.5,3.0,0.0
9,10,Chase Brown (CIN),15.9,0.3,3.0,1.9,9.0,0.0,0.1,0.3,0.3,1.7,0.0


### Model training

In [7]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 0.4605796126596308
Test Set MSE: 0.3792964786847713
Random Forest Test Set MSE: 0.055098481427294406


/tmp/ipykernel_13517/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [8]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("RB_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,Y/A,G,FPTS,FPTS/G,ATT/game,YDS/game,20+/game,TD/game,REC/game,TGT/game,YDS.1/game,TD.1/game
Rank,,,,,,,,,,,,,,,,
2,1,Saquon Barkley (PHI),10.00,1,5.8,16,355.3,22.2,21.6,125.3,1.1,0.8,2.1,2.7,17.4,0.1
4,2,Derrick Henry (BAL),9.18,2,5.9,17,336.4,19.8,19.1,113.0,1.1,0.9,1.1,1.3,11.4,0.1
1,3,Jahmyr Gibbs (DET),8.97,-2,5.6,17,362.9,21.3,14.7,83.1,0.8,0.9,3.1,3.7,30.4,0.2
3,4,Bijan Robinson (ATL),8.40,-1,4.8,17,341.7,20.1,17.9,85.6,0.3,0.8,3.6,4.2,25.4,0.1
12,5,Jonathan Taylor (IND),8.23,7,4.7,14,244.7,17.5,21.6,102.2,0.8,0.8,1.3,2.2,9.7,0.1
9,6,Alvin Kamara (NO),7.78,3,4.2,14,265.3,19.0,16.3,67.9,0.2,0.4,4.9,6.4,38.8,0.1
17,7,Joe Mixon (HOU),7.58,10,4.1,14,240.5,17.2,17.5,72.6,0.6,0.8,2.6,3.7,22.1,0.1
6,8,Josh Jacobs (GB),7.44,-2,4.4,17,293.1,17.2,17.7,78.2,0.4,0.9,2.1,2.5,20.1,0.1
5,9,De'Von Achane (MIA),7.37,-4,4.5,17,299.9,17.6,11.9,53.4,0.3,0.4,4.6,5.1,34.8,0.4
